In [16]:
"""
Initial setup:
1) create virtualenv project using pycharm

2) install the following libraries to virtualenv:
pip install numpy
pip install pandas
pip install requests
pip install beautifulsoup4
pip install lxml
"""

'\nInitial setup:\n1) create virtualenv project using pycharm\n\n2) install the following libraries to virtualenv:\npip install numpy\npip install pandas\npip install requests\npip install beautifulsoup4\npip install lxml\n'

In [1]:
import requests  # library to send requests to web site(krisha.kz)
from bs4 import BeautifulSoup as bs  # library to copy all html-code
import time
import math
import re
import json
from lxml import html
import regex
import pandas as pd

In [2]:
headers = {
    'accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

session = requests.Session()

def get_all_urls(catalog_page_url):
    result = [catalog_page_url]
    url = catalog_page_url
    all_data = get_page(url)
    pages = math.ceil(all_data["total"] / all_data["limit"])
    for i in range(2, pages):
        url = catalog_page_url + '?page={i}'.format(i=i)
        if url not in result:
            result.append(url)
    return result


def get_page(url):
    request = session.get(url, headers=headers)
    tree = html.fromstring(request.content)
    script = tree.xpath('//script[contains(., "BACKEND.components.catalogGrid")]/text()')[0]
    data = regex.compile(r"\{(?:[^{}]|(?R))*\}").findall(str(script))[0]
    all_data = json.loads(data)
    return all_data


def parse_kaspi_pages(urls):
    all_cards = []
    c = -1000 # abort if there is too many pages
    for url in urls:
        time.sleep(1)
        print("parsing: " + url)
        all_data = get_page(url)
        cards = all_data["cards"]
        for card in cards:
            card.pop("monthlyInstallment", None)
            card.pop("previewImages", None)
            card.pop("category", None)
            card.pop("unit", None)
        all_cards += cards
        c += 1
        if c > 2:
            break
    return all_cards

base_url = 'https://kaspi.kz/shop/c/notebooks/'
items = parse_kaspi_pages(urls = get_all_urls(base_url))
session.close()
df = pd.json_normalize(items)
df.to_csv('kaspi_more_data.csv', encoding='utf-8', index=False)

parsing: https://kaspi.kz/shop/c/notebooks/
parsing: https://kaspi.kz/shop/c/notebooks/?page=2
parsing: https://kaspi.kz/shop/c/notebooks/?page=3


In [22]:
import csv

items = []
with open('Kaspi_data.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    initial = True
    for row in spamreader:
        if not initial:
            title = row[0].split(" ")
            price = re.sub('\D', '', row[1])
            rating = row[2]

            items.append({
                'brand': title[0],
                'color': title[-1],
                'full_name': ' '.join(title[:-1]),
                'price': price,
                'rating': rating,
            })
        else:
            initial = False

print(items[0])
def files_writer(items):
    # with open('HeadHunter.csv', 'a', encoding='utf-8') as file:
    with open(r"Kaspi_data.csv", "w", encoding='utf-8') as file:
        a_pen = csv.writer(file)
        a_pen.writerow(('brand', 'color', 'full_name', 'price', 'rating'))
        for item in items:
            a_pen.writerow((item['brand'], item['color'], item['full_name'], item['price'], item['rating']))

files_writer(items)

{'brand': 'Lenovo', 'color': 'серый', 'full_name': 'Lenovo V14-ADA 82C6S03900', 'price': '184800', 'rating': '297'}
